In [1]:
#Import Dependencies
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

In [2]:
#Read in data
exo = pd.read_csv('../data/clean_data.csv')
#Drop vestial indes
exo.drop('Unnamed: 0', axis=1, inplace=True)
exo.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,170.538750,0.146,2.95750,615.8,...,793.0,93.59,35.8,1.0,5455.0,4.467,0.927,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,...,443.0,9.11,25.8,2.0,5455.0,4.467,0.927,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,...,638.0,39.30,76.3,1.0,5853.0,4.544,0.868,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,...,1395.0,891.96,505.6,1.0,5805.0,4.564,0.791,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,...,1406.0,926.16,40.9,1.0,6031.0,4.438,1.046,288.75488,48.226200,15.509


In [3]:
#Set data and target
data = exo.drop('koi_disposition', axis=1)
feature_names = data.columns
target = exo['koi_disposition']
target_names = ['CONFIRMED', 'FALSE POSITIVE', 'CANIDATE']

In [4]:
#Categorize target
label_encoder = LabelEncoder()
label_encoder.fit(target)
encoded_target = label_encoder.transform(target)

In [5]:
#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, encoded_target, random_state=1)

In [6]:
#Set training and testing categories
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [7]:
#Scale data
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
#Set model
model = Sequential()
#Add layers to model
model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 22,603
Trainable params: 22,603
Non-trainable params: 0
_________________________________________________________________


In [9]:
#Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
#Fit the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 6708 samples
Epoch 1/100
6708/6708 - 1s - loss: 0.4685 - accuracy: 0.7689
Epoch 2/100
6708/6708 - 1s - loss: 0.3533 - accuracy: 0.8201
Epoch 3/100
6708/6708 - 0s - loss: 0.3387 - accuracy: 0.8290
Epoch 4/100
6708/6708 - 0s - loss: 0.3251 - accuracy: 0.8372
Epoch 5/100
6708/6708 - 0s - loss: 0.3212 - accuracy: 0.8377
Epoch 6/100
6708/6708 - 0s - loss: 0.3159 - accuracy: 0.8433
Epoch 7/100
6708/6708 - 0s - loss: 0.3042 - accuracy: 0.8475
Epoch 8/100
6708/6708 - 0s - loss: 0.2939 - accuracy: 0.8518
Epoch 9/100
6708/6708 - 0s - loss: 0.2936 - accuracy: 0.8521
Epoch 10/100
6708/6708 - 0s - loss: 0.2865 - accuracy: 0.8561
Epoch 11/100
6708/6708 - 0s - loss: 0.2831 - accuracy: 0.8566
Epoch 12/100
6708/6708 - 0s - loss: 0.2817 - accuracy: 0.8615
Epoch 13/100
6708/6708 - 0s - loss: 0.2749 - accuracy: 0.8615
Epoch 14/100
6708/6708 - 0s - loss: 0.2704 - accuracy: 0.8649
Epoch 15/100
6708/6708 - 0s - loss: 0.2666 - accuracy: 0.8660
Epoch 16/100
6708/6708 - 0s - loss: 0.2654 - accuracy: 0.

In [11]:
#Test model on various samples
for i in range(1, 26):
    X_train, X_test, y_train, y_test = train_test_split(data, encoded_target, random_state=i)
    y_test_categorical = to_categorical(y_test)
    X_test_scaled = X_scaler.transform(X_test)
    model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
    print(f"DNN test {i} - Loss: {model_loss}, Accuracy: {model_accuracy}")

2237/2237 - 0s - loss: 1.1601 - accuracy: 0.8373
DNN test 1 - Loss: 1.160099078141316, Accuracy: 0.8372820615768433
2237/2237 - 0s - loss: 0.3036 - accuracy: 0.9517
DNN test 2 - Loss: 0.3036477334087962, Accuracy: 0.9517210721969604
2237/2237 - 0s - loss: 0.2573 - accuracy: 0.9531
DNN test 3 - Loss: 0.2572804035474852, Accuracy: 0.953062117099762
2237/2237 - 0s - loss: 0.2405 - accuracy: 0.9549
DNN test 4 - Loss: 0.24049757216184012, Accuracy: 0.9548502564430237
2237/2237 - 0s - loss: 0.2312 - accuracy: 0.9504
DNN test 5 - Loss: 0.2311934536388731, Accuracy: 0.9503799676895142
2237/2237 - 0s - loss: 0.2488 - accuracy: 0.9508
DNN test 6 - Loss: 0.24884957483951728, Accuracy: 0.9508270025253296
2237/2237 - 0s - loss: 0.4583 - accuracy: 0.9522
DNN test 7 - Loss: 0.45832808475623343, Accuracy: 0.9521681070327759
2237/2237 - 0s - loss: 0.2155 - accuracy: 0.9616
DNN test 8 - Loss: 0.21548716151386593, Accuracy: 0.9615556597709656
2237/2237 - 0s - loss: 0.4283 - accuracy: 0.9481
DNN test 9 - 

In [12]:
#Save model
model.save('DNN.h5')